# [AI达人创造营第二期] 从电影推荐系统出发了解基于用户的协同过滤算法

## 1. 项目背景介绍
### 1.1 协同过滤算法
协同过滤算法是推荐算法领域中基础但非常重要的部分, 它从1992年开始投入推荐算法的研究过程中, 并在AMAZON等大型电子商务的推荐系统中起到了非常出色的效果.

协同过滤算法可以被分为基于用户的协同过滤算法以及基于项目的协同过滤算法. 本项目将从电影推荐系统的简单构建出发来介绍基于用户的协同过滤算法

### 1.2 以用户为基础（User-based）的协同过滤
用相似统计的方法得到具有相似爱好或者兴趣的相邻用户, 所以称之为以用户为基础（User-based）的协同过滤或基于邻居的协同过滤(Neighbor-based Collaborative Filtering). 

![](https://ai-studio-static-online.cdn.bcebos.com/58ff5bf39564492ca5b5c989d8078bddf0e7b7594fb645a497cb386be3d837c7)

基本方法步骤：
1. 收集用户信息

收集可以代表用户兴趣的信息, 一般的网站系统使用评分的方式或是给予评价, 这种方式被称为“主动评分”, 另外一种是“被动评分”, 是根据用户的行为模式由系统代替用户完成评价. 不需要用户直接打分或输入评价数据. 电子商务网站在被动评分的数据获取上有其优势, 用户购买的商品记录是相当有用的数据.

2. 最近邻搜索(Nearest neighbor search, NNS)

以用户为基础（User-based）的协同过滤的出发点是与用户兴趣爱好相同的另一组用户, 就是计算两个用户的相似度. 例如：查找n个和A有相似兴趣用户, 把他们对M的评分作为A对M的评分预测. 一般会根据数据的不同选择不同的算法, 较多使用的相似度算法有Pearson Correlation Coefficient、Cosine-based Similarity、Adjusted Cosine Similarity.
    
3. 产生推荐结果

有了最近邻集合, 就可以对目标用户的兴趣进行预测，产生推荐结果。依据推荐目的的不同进行不同形式的推荐, 较常见的推荐结果有Top-N 推荐和关系推荐。Top-N 推荐是针对个体用户产生, 对每个人产生不一样的结果, 例如：通过对A用户的最近邻用户进行统计, 选择出现频率高且在A用户的评分项目中不存在的，作为推荐结果。关系推荐是对最近邻用户的记录进行关系规则(association rules)挖掘.

## 2. 数据介绍
模型决定复现经典的协同过滤算法(CF), 在本项目中, 拟使用Movielens的ml-latest数据集来完成一个简单的电影推荐系统.

MovieLens数据集包含多个用户对多部电影的评级数据, 也包括电影元数据信息和用户属性信息.

这个数据集经常用来做推荐系统, 机器学习算法的测试数据集. 尤其在推荐系统领域, 很多著名论文都是基于这个数据集的.

本文采用的是MovieLens中的ml-latest数据集.

### 2.1 解压数据集及导入依赖包
项目中导入了m1-latest数据集

In [129]:
!unzip -oq data/data101354/ml-latest.zip -d data/
print('解压成功!')

解压成功!


In [130]:
import pandas as pd
import paddle
import numpy as np

### 2.2 数据集的读取
在读取数据集时, 由于评价时间在统计过程中没有用到, 所以不读取. 为了快速看到训练效果, 只取前10w个数据作为小量数据集

In [131]:
dtype = {'userId': np.int64, 'movieId': np.int64, 'rating': np.float32}
# 由于评价时间在统计过程中没有用到, 所以不读取. 为了缩短训练时间, 只取前10w个数据作为小量数据集
ratings_data = pd.read_csv(r'data/ml-latest/ratings.csv', dtype=dtype, usecols=[0,1,2], nrows=100000)
print('success!')

success!


### 2.3 数据的可视化与处理
在完成数据的读取后, 对数据集的基本信息进行简单的了解, 以及转变数据集的储存格式

In [132]:
# 数据的基本结构
print(ratings_data.head())
# 数据集的信息
print(ratings_data.describe())

   userId  movieId  rating
0       1      307     3.5
1       1      481     3.5
2       1     1091     1.5
3       1     1257     4.5
4       1     1449     4.5
              userId        movieId         rating
count  100000.000000  100000.000000  100000.000000
mean      507.615380   18282.999630       3.507395
std       303.510237   35015.296487       1.103155
min         1.000000       1.000000       0.500000
25%       239.000000    1079.000000       3.000000
50%       491.000000    2580.000000       4.000000
75%       780.000000    6934.000000       4.000000
max      1041.000000  192579.000000       5.000000


在看完基本的数据集信息后, 为了能够更直观地看到用户和电影之间的关系, 以及更好地调用同一个用户的电影评分以及同一部电影不同用户给出的评分, 现将数据集转化为透视表的形式, 将评价信息转化为用户对电影的评分矩阵

In [133]:
# 构建透视表
ratings_matrix = ratings_data.pivot_table(index='userId', columns='movieId', values='rating')
# 透视表概览
ratings_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,189363,189399,190017,190085,192005,192081,192215,192219,192225,192579
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4.0,NaN,NaN,2.0,4.5,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1038,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. 模型构建

### 3.1 相似度的计算
在协同过滤算法中, 经常使用的相似度有三种, 分别为余弦相似度, 皮尔逊(Pearson)相似度, 以及杰卡德(Jacaard)相似度.


- 余弦相似度, Pearson相似度
	- 余弦相似度: 
   $$sim(a,b) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}| \times |\vec{b}|}$$
   ![](https://ai-studio-static-online.cdn.bcebos.com/67184bf7b0d84c6799d0b42dff459437c9f04dc2b93c4c979119de47e5422e07)
   - Pearson相似度:
   $$corr(a,b) = \frac{\sum _{i} (r_{ai}-\overline{r_a})(r_{b	i}-\overline{r_b})}{\sqrt{\sum _{i}(r_{ai}-\overline{r_a})^2 \sum _{i}(r_{b	i}-\overline{r_b})^2}}$$
	- 都为向量的余弦角值
   - Pearson相似度会对向量的每一个分量做中心化处理
   - 相对于余弦相似度, Pearson相似度还考虑每一个向量的长度, 因此Pearson更加常用
   - 在评价数据是连续分布的情况下, 常使用余弦相似度以及Pearson相似度
- Jaccard相似度
	- 计算方法: $sim(a,b) = \frac{交集}{并集}$
   - 在计算评分数据为布尔值的情况下, 使用Jaccard相似度



对电影评分的预测我们使用Pearson相似度来作为用户之间的相似度.


In [134]:
# 计算pearson_similarity
similarity = ratings_matrix.T.corr()
similarity

userId,1,2,3,4,5,6,7,8,9,10,...,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,NaN,NaN,0.503227,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN
4,0.503227,NaN,1.0,1.000000,-0.089127,0.482844,0.0,0.576248,NaN,0.258401,...,-0.866025,0.175456,0.645497,0.277753,NaN,-0.378968,-0.270031,0.484860,-0.152101,0.215078
5,NaN,NaN,NaN,-0.089127,1.000000,-0.577350,NaN,NaN,NaN,0.205196,...,NaN,-0.279953,0.000000,0.318242,NaN,NaN,NaN,0.490098,-0.131832,-0.461538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,NaN,NaN,NaN,-0.378968,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
1038,NaN,NaN,NaN,-0.270031,NaN,NaN,NaN,0.534522,NaN,NaN,...,NaN,NaN,NaN,-1.000000,NaN,NaN,1.000000,NaN,NaN,NaN
1039,NaN,NaN,NaN,0.484860,0.490098,0.401955,NaN,NaN,NaN,0.301511,...,NaN,-0.526152,1.000000,0.565205,NaN,NaN,NaN,1.000000,0.476731,0.000000


### 3.2 构建预测模型
在得到了用户之间的相似度矩阵后, 我们就可以开始构建对于某个指定的用户对电影的喜好程度的预测了. 以下是实现思路:
1. 对相似度矩阵先进行处理, 去除无关用户和与目标用户负相关的用户, 得到初步的相似用户similar_users
2. 在相似用户中进行筛选, 只留下那些只看过我们目标电影的用户作为最终参与预测的用户群体final_similar_users
3. 根据评分预测的公式
$$pred(user, movie) = \frac{\sum_{v \in U} sim(user, movie) * r_{vi}}{\sum_{v \in U} |sim(user, movie)|}$$
我们可以计算出预测分数

In [135]:
# 构建指定用户对指定电影的评分预测
def predict(user, movie, ratings_matrix, similarity):
    # 找到和目标user相关的用户
    similar_users = similarity[user].drop([user]).dropna()
    # 去除掉负相关的干扰项
    similar_users = similar_users.where(similar_users>0).dropna()
    # 找到其中评价过目标电影的用户
    idx = ratings_matrix[movie].dropna().index & similar_users.index
    # 得到最终这些相似用户的相似度
    final_similar_users = similar_users.loc[list(idx)]
    # 初始化评分预测公式的分子分母
    sum_up = 0
    sum_down = 0
    for sim_user, similarity in final_similar_users.iteritems():
        # 相似用户的评分数据
        sim_user_rated_movies = ratings_matrix.loc[sim_user].dropna()
        # 相似用户对目标电影的评分
        sim_user_rating_for_movie = sim_user_rated_movies[movie]
        sum_up += similarity*sim_user_rating_for_movie
        sum_down += similarity
    
    # 计算预测评分
    predict_rating = sum_up/sum_down
    return predict_rating


### 3.3 构建排序模型
通过之前计算得到的评分预测模型, 我们可以通过计算指定用户对电影的评分预测来得到电影评分预测的排序, 从而给出针对特定用户的电影推荐.


在这个过程中, 首先我们将构建一个对所有电影进行评分预测的函数, 这其中会包含对极端数据的处理:
- 不会再次预测目标用户已经观看过的电影.
- 不会给用户推荐评分数量过少的"冷门"电影.
通过筛选掉这些电影之后, 我们就会得到对一个特定用户来说的电影评分预测清单.


利用这个预测清单既可以轻松得到推荐电影的movieId

In [136]:
# 对一个指定用户的所有电影进行评分预测
def predict_all(user, ratings_matrix, similarity, filter_rule=None):
    # 添加过滤条件简化数据集中的冗余数据
    if not filter_rule:
        # 获取电影Id索引
        movie_idx = ratings_matrix.columns
    elif filter_rule == ['unhot','rated']:
        # 去除用户已经看过的电影
        user_ratings = ratings_matrix.iloc[user]
        # 判断已经有过评分的电影
        _ = user_ratings<=5
        idx1 = _.where(_ == False).dropna().index

        # 去除冷门电影, 热门电影的判断标准暂定为被观看(评分)超过十次
        count = ratings_matrix.count()
        idx2 = count.where(count>10).dropna().index

        movie_idx = set(idx1) & set(idx2)
    else:
        raise Exception('无效过滤参数')
        
    # 进入循环
    for movie in movie_idx:
        try:
            rating = predict(user, movie, ratings_matrix, similarity)
        except Exception as e:
            pass
        else:
            yield user, movie, rating


In [137]:
# 给出前n项推荐的电影ID
def rank(user, n):
    results = predict_all(user, ratings_matrix, similarity, filter_rule=['unhot','rated'])
    # 根据分数进行降序排序, 然后输出前n项
    return sorted(results, key=lambda x: x[2], reverse = True)[:n]

### 3.4 结合movies.csv数据集输出电影名称
在得到了推荐的电影的movieId后, 可以通过movies,csv来读取电影名称最后输出推荐的电影.

In [138]:
# 读取movies数据集
movie_names = pd.read_csv(r'data/ml-latest/movies.csv')
print('success!')

success!


In [139]:
# 电影id与名称对应概览
# print(movie_names.head())
movie_id = 1
print(movie_names.where(movie_names['movieId']==movie_id).dropna().values[0][1])

Toy Story (1995)


In [140]:
# 输出推荐的电影的电影名称
def get_movie_name(rank, movie_names):
    count = 0
    print(f'推荐的电影按推荐力度排列如下:')
    # rank函数返回的是一个二维数组, 其中每一项的第二个数据为推荐的movieId
    for item in rank:
        movie_id = item[1]
        count += 1
        print(f'{count}.', movie_names.where(movie_names['movieId']==movie_id).dropna().values[0][1])

In [141]:
# 电影推荐实例测试
user_id = 100
# 输出top10推荐电影
get_movie_name(rank(user_id,10), movie_names)

推荐的电影按推荐力度排列如下:
1. Thin Man, The (1934)
2. Inherit the Wind (1960)
3. Strangers on a Train (1951)
4. 12 Angry Men (1957)
5. Doctor Zhivago (1965)
6. Roger & Me (1989)
7. Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
8. Shawshank Redemption, The (1994)
9. Treasure of the Sierra Madre, The (1948)
10. Seven Samurai (Shichinin no samurai) (1954)


## 4. 总结与升华
协同过滤算法是推荐算法领域的重要部分, 它有许多优点, 比如能够过滤机器难以自动内容分析的信息, 避免了内容分析的不完全或不精确; 并且能够基于一些复杂的, 难以表述的概念(如信息质量, 个人品味)进行过滤等等.

但是协同过滤算法同样有着许多不足:
1. 对于新用户的推荐效果就较差, 由于新用户的评价数据较少, 就很难确定用户的准确的相似用户群体, 因此也难以给出准确的判断
2. 由于推荐系统的应用场景大部分都是在具有非常庞大的项目数量的基础上的, 因此用户评价数据的稀疏性也会成为一个值得关注的问题

对于协同过滤算法中的每一个细节, 我们也都可以思考优化的方向, 比如相似度的计算, 数据的读取使用的方法, 相似度矩阵的储存和读取等等. 当下本项目使用的仅仅是节选的10万的数据集, 在面对实际应用时大体量的数据场景下, 这些问题的优化就会起到举足轻重的作用.

In [142]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data101354  ml-latest


In [143]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [144]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/69/bf/f0f194d3379d3f3347478bd267f754fc68c11cbf2fe302a6ab69447b1417/beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/72/a6/fd01694427f1c3fcadfdc5f1de901b813b9ac756f0806ef470cfed1de281/soupsieve-2.3.1-py3-none-any.whl (37 kB)
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.


In [145]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 